# Taxable revenues in Barcelona

Open Data Barcelona provides lots of fun data about our city. You can access it here: https://opendata-ajuntament.barcelona.cat

We will study the mean tax revenues per neighborhood ("barri") in the years 2016 and 2015. Tax revenues are a proxy for (taxable) income, so we're in a way looking at how income varies across the city. 

Barcelona is divided in districts and each district is divided in 'barri' or neighborhood. Each neighborhood of Barcelona is divided in several census tracts. In '2016_renda.csv' we have the mean tax revenues (tax revenues / number of tax returns) in each census tract of the city. 

The columns are in Catalan, so here's a quick explanation in English: 

Any = Year  
Codi_Districte = District Code  
Nom_Districte = District Name  
Codi_Barri = Neighborhood Code  
Nom_Barri = Neighborhood Name  
Seccio_Censal = Census Tract Number  
Import_Euros = Mean tax revenues

First, import the data in "2016_renda.csv" into a Pandas DataFrame.

In [1]:
import pandas as pd

#Your code here.

Get the 5 neighborhoods (barris) with the highest average mean tax revenues.

In [2]:
#Your code here.

Get the difference in mean revenue between the poorest census tract and the richest, within each district.

You should return a DataFrame with 2 columns:
the district name and the difference in mean revenues.

In [3]:
#Your code here.

## Planning Your Attack

One way to make your code easier to understand is to ensure that your code can be read on two levels: one "declarative" level, where someone can read (or write) *what* will happen and another "imperative level", where someone can read (or write!) *how* the thing is happening.

Data preparation often involves a "pipeline", a uni-directional flow of transformations where the data is moved, one step at a time, towards the final format.

It's important to plan ahead when you design your pipline.

One way to make a plan is to start from the final goal, and write out the following statement: 

1. "If I had ________ (INPUT), then it would be easy to make [FINAL GOAL], I would just need to ________ (step)."

Where you should think of INPUT as "data ______ in data structure ______".

That will be the final step of your pipeline. Now repeat the statement, with the FINAL GOAL being replaced with the INPUT of the previous step: 

2. "If I had ________ (INPUT), then it would be easy to make [PREVIOUS INPUT], I would just need to ________ (step)."

Let's see an example of this method of planning by working out an exercise:


Your goal will be the following: 

We want to understand the income variation (or "spatial inequality") within each neighborhood. However, each neighborhood has a different size.
Larger neighborhoods will naturally have a greater variation, even if there isn't great variation between one block and the next, which is what
we want to understand with spatial inequality.

To deal with this, we will apply the naive solution of simply using the number of census tracts as a proxy for the "physical size" of the neighborhood. We will then divide the income gap (difference between lowest and highest income tract) within each neighborhood by the number of tracts as a way to control for size. This will be our measure of spatial inequality.

Your job is to return a dataframe sorted by our measure of spatial inequality where any neighborhood with one tract (0 inequality) has been removed.


We will try to lay out a plan to solve the problem with the process we just went over:

1. If I had a *column with
   the income gap divided by the number of tracts*
   then it would be easy to *get the neighborhoods with 
   highest and lowest spatial inequality*, I 
   would just need to *sort the dataframe by that
   column*.

2. If I had *A. a column for the income gap and 
   B. a column for the number of tracts in a neighborhood*
   then it would be easy to get a *column with
   the income gap divided by the number of tracts*
   I would just need to *divide the first one column by the second*. 

3. If I had *the raw data*, then it would be easy to add
   *a column with the number of tracts*, I would just need
   to *count the number of tracts per neighborhood*.

4. If I had *the raw data*, then it would be easy to add
   *a column with the income gap*, I would just need to
   *calculate the income difference between tracts in each 
   neighborhood*. 

Now we can use this outline to write a declarative pipeline
function (in the opposite order of the steps we wrote): 

In [4]:
def spatial_inequality_in_barcelona(df):
    df = add_income_diff_for_barris(df) #4
    df = add_num_tracts_per_barri(df) #3
    df = add_inequality(df) #2
    return inequality_by_barri(df) #1

In the next exercises, you will write each of the function of the pipeline above and in the end, use this function to compare barris based on their spatial inequality.

Write the function: "add_income_diff_for_barris"

HINT: Make sure the returned dataframe is the same size as the original!


In [5]:
def add_diff(df):
    gap = get_inequality(df)
    return df.assign(gap=gap)

def add_income_diff_for_barris(df):
    return df.groupby('Nom_Barri') \
             .apply(add_diff) \
             .reset_index(drop=True)

Write the function: "add_num_tracts_per_barri"

In [6]:
def add_num_tracts_per_barri(df):
    return df.groupby('Nom_Barri') \
             .apply(lambda df: df.assign(num_tracts = df.shape[0])) \
             .reset_index(drop=True)

Write the function: "add_inequality"

In [7]:
def add_inequality(df):
    return df.groupby('Nom_Barri') \
             .apply(lambda df: df.assign(inequality = df.gap/df.num_tracts)) \
             .reset_index(drop=True)


Write the function "inequality_by_barri". Note that this function should check that the dataframe has the same number of rows as number of barris (i.e. one barri per row). Also if some barris have an inequality at 0, remove them.

In [8]:
def inequality_by_barri(df):
    return df.drop_duplicates('Nom_Barri') \
             .drop(columns = ['Seccio_Censal']) \
             .sort_values('inequality') \
             .pipe(lambda df: df[df.inequality != 0])

Try out the function we wrote out in the planning phase, `spatial_inequality_in_barcelona`, does it work when given the raw data?

In [9]:
#Your code here.

Now let's "refactor". "Refactoring" means rewriting the code without changing the functionality. What we wrote works and is readable. 

But maybe breaking it down into so many separate steps, while didactic, isn't the most efficient. You probably grouped by "Nom_Barri" at least 3 separate times!

Try to rewrite the function spatial_inequality_in_barcelona to be more efficient (to only groupby Nom_Barri once for example) and shorter.

In [11]:
def add_inequality(df):
    gap = df.Import_Euros.max() - df.Import_Euros.min()
    sections = df.shape[0]
    return df.assign(gap=gap, 
                     sections=sections, 
                     inequality=gap/sections)

def spatial_inequality_in_barcelona(df):
    return df.groupby('Nom_Barri') \
             .apply(add_inequality) \
             .reset_index(drop=True) \
             .sort_values('inequality') \
             .pipe(lambda df: df[df.gap != 0]) \
             [['Nom_Barri', 'gap', 'sections', 'inequality']]  

#spatial_inequality_in_barcelona(df)